<a href="https://colab.research.google.com/github/kodeu/trading-prophet/blob/prophet_iter1/prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files

dataext = []
data = []
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=filename, length=len(uploaded[filename])))
filename = list(uploaded.keys())[0]  # Adjust as per actual file name if needed


# Load the JSON data from the file
with open(filename, 'r') as file:
    rawdata = json.load(file)

rawdata = np.array(rawdata)
rawdata = rawdata.astype(np.float64)

print(rawdata.shape)
print(rawdata)

Saving klines_btcusdt_1000.json to klines_btcusdt_1000.json
User uploaded file "klines_btcusdt_1000.json" with length 187508 bytes
(1000, 12)
[[1.62967680e+12 4.92392200e+04 5.05000000e+04 ... 2.62189751e+04
  1.30781764e+09 0.00000000e+00]
 [1.62976320e+12 4.94888500e+04 4.98600000e+04 ... 2.47029234e+04
  1.20389965e+09 0.00000000e+00]
 [1.62984960e+12 4.76740100e+04 4.92643000e+04 ... 2.17968777e+04
  1.04971210e+09 0.00000000e+00]
 ...
 [1.71581760e+12 6.62065100e+04 6.67520100e+04 ... 1.56451126e+04
  1.02896231e+09 0.00000000e+00]
 [1.71590400e+12 6.52352100e+04 6.74512000e+04 ... 1.33010953e+04
  8.83129353e+08 0.00000000e+00]
 [1.71599040e+12 6.70240000e+04 6.70958800e+04 ... 1.09431740e+02
  7.32935466e+06 0.00000000e+00]]
